In [1]:
import numpy as np
import tensorflow as tf
import os

from keras.utils import np_utils
import cv2


Using TensorFlow backend.


In [2]:

(train_features, train_labels), (test_features, test_labels) = tf.keras.datasets.cifar10.load_data()
num_train, img_channels, img_rows, img_cols =  train_features.shape
num_test, _, _, _ =  test_features.shape
num_classes = len(np.unique(train_labels))

170500096/170498071 [==============================] - 11s 0us/step


In [0]:
train_features = train_features.astype('float32')/255
test_features = test_features.astype('float32')/255
# convert class labels to binary class labels
train_labels = np_utils.to_categorical(train_labels, num_classes)
test_labels = np_utils.to_categorical(test_labels, num_classes)

In [0]:
def get_cutout_eraser_and_random_crop(p=0.5,s_l=0.05,s_h=0.3,r_1=0.3,r_2=1/0.3,max_erasers_per_image=1,pixel_level=True,random_crop_size=(32,32),padding_pixels=4):
  
  assert max_erasers_per_image>=1 
  def eraser(input_img):
        v_l = np.min(input_img)
        v_h = np.max(input_img)
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img
        mx = np.random.randint(1,max_erasers_per_image+1)
        for i in range(mx):
          while True:
              s = np.random.uniform(s_l, s_h) * img_h * img_w
              r = np.random.uniform(r_1, r_2)
              w = int(np.sqrt(s / r))
              h = int(np.sqrt(s * r))
              left = np.random.randint(0, img_w)
              top = np.random.randint(0, img_h)

              if left + w <= img_w and top + h <= img_h:
                  break

          if pixel_level:
              c = np.random.uniform(v_l, v_h, (h, w, img_c))
          else:
              c = np.random.uniform(v_l, v_h)

          input_img[top:top + h, left:left + w, :] = c
        return input_img

    
  def random_crop(input_image):
    assert input_image.shape[2]==3

    #pad for 4 pixels
    img = cv2.copyMakeBorder(input_image,padding_pixels,padding_pixels,padding_pixels,padding_pixels,cv2.BORDER_REPLICATE)
    height , width =img.shape[0],img.shape[1]
    dy,dx = random_crop_size
    x = np.random.randint(0,width - dx + 1)
    y = np.random.randint(0,height - dy + 1)
    return img[y:(y+dy),x:(x+dx),:]

  def preprocess_image(input_image):
    return eraser(random_crop(input_image))
  
  return preprocess_image
      

In [0]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=0.5,featurewise_center=True, featurewise_std_normalization=True,preprocessing_function=get_cutout_eraser_and_random_crop())
datagen.mean = np.array([0.4914, 0.4822, 0.4465], dtype=np.float32).reshape((1,1,3)) # ordering: [R, G, B]
datagen.std = np.array([0.2023, 0.1994, 0.2010], dtype=np.float32).reshape((1,1,3)) # ordering: [R, G, B]
#datagen.fit(train_features)
train_generator = datagen.flow(train_features,train_labels,batch_size=128)

In [0]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
test_datagen.fit(test_features)
test_generator = test_datagen.flow(test_features,test_labels,batch_size=128)

#Resnet Model Creation

In [7]:
# import time, math
# def init_pytorch(shape, dtype=tf.float32, partition_info=None):
#   fan = np.prod(shape[:-1])
#   bound = 1 / math.sqrt(fan)
#   return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

initializer = tf.keras.initializers.glorot_normal(seed=None)

W0807 09:57:21.262352 140373475358592 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1288: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
def ResNetBlock(input_layer, channels,stride=1):
  
  bn_1 = tf.keras.layers.BatchNormalization(momentum=0.9,epsilon=1e-5)(input_layer)
  activation_layer_b1 = tf.keras.layers.Activation('relu')(bn_1)
  if(stride==2):
    print("here")
    block_layer_1 = tf.keras.layers.Conv2D(channels, (3,3),dilation_rate=(1,1) ,padding='same',kernel_initializer=initializer,use_bias=False)(activation_layer_b1)
    block_layer_1= tf.keras.layers.MaxPooling2D()(block_layer_1)
  else:
    block_layer_1 = tf.keras.layers.Conv2D(channels, (3,3), padding='same',dilation_rate=(1,1),kernel_initializer=initializer,use_bias=False)(activation_layer_b1)
  
  bn_2 = tf.keras.layers.BatchNormalization(momentum=0.9,epsilon=1e-5)(block_layer_1)
  activation_layer_b2 = tf.keras.layers.Activation('relu')(bn_2) 
  block_layer_2 = tf.keras.layers.Conv2D(channels, (3,3), padding='same',dilation_rate=(1,1),kernel_initializer=initializer,use_bias=False)(activation_layer_b2)
   
  
  return block_layer_2
  

In [9]:
# from tf.keras.layers import Input, add, GlobalAveragePooling2D, Dense
#from tf.keras.models import Model

inputs = tf.keras.layers.Input(shape=(32, 32, 3))

x1 = tf.keras.layers.Conv2D(32 ,(3, 3),padding='same',kernel_initializer=initializer,use_bias=False)(inputs)   #32x32 
activation_x1 = tf.keras.layers.Activation('relu')(x1)
bn1 = tf.keras.layers.BatchNormalization(momentum=0.9,epsilon=1e-5)(activation_x1)

# x2 = tf.keras.layers.Conv2D(32, (3, 3),padding='same',kernel_initializer=initializer)(bn1)   #32x32 
# activation_x2 = tf.keras.layers.Activation('relu')(x2)
# bn2 = tf.keras.layers.BatchNormalization(momentum=0.9,epsilon=1e-5)(activation_x2)

# x3 = tf.keras.layers.Conv2D(32, (3, 3),padding='same')(activation_x2)   #32x32 
# activation_x3 = tf.keras.layers.Activation('relu')(x3)
# bn3 = tf.keras.layers.BatchNormalization(momentum=0.9,epsilon=1e-5)(activation_x3)

##block 1



blk1 = ResNetBlock(bn1,32)  ##32x32
z1 = tf.keras.layers.add([blk1,bn1])

blk1_c = ResNetBlock(z1,32)
z1_c = tf.keras.layers.add([blk1_c,z1])



##block 2

blk2 = ResNetBlock(z1_c,64,stride=2)
mx_1= tf.keras.layers.MaxPooling2D()(z1_c)
one_blk = tf.keras.layers.Conv2D(64, (1, 1), padding='same',kernel_initializer=initializer,use_bias=False)(mx_1)
z2 = tf.keras.layers.add([blk2,one_blk])

blk2_c = ResNetBlock(z2,64)
z2_c = tf.keras.layers.add([blk2_c,z2])



##block3

blk3 = ResNetBlock(z2_c,128,stride=2)
mx_2= tf.keras.layers.MaxPooling2D()(z2_c)

one_blk_1 = tf.keras.layers.Conv2D(128, (1, 1), padding='same',kernel_initializer=initializer,use_bias=False)(mx_2)
z3 = tf.keras.layers.add([blk3,one_blk_1])

blk3_c = ResNetBlock(z3,128)
z3_c = tf.keras.layers.add([blk3_c,z3])


##block4

blk4 = ResNetBlock(z3_c,256,stride=2)
mx_3= tf.keras.layers.MaxPooling2D()(z3_c)

one_blk_2 = tf.keras.layers.Conv2D(256, (1, 1), padding='same')(mx_3)
z4 = tf.keras.layers.add([blk4,one_blk_2])

blk4_c = ResNetBlock(z4,256)
z4_c = tf.keras.layers.add([blk4_c,z4])

avg_pool_layer = tf.keras.layers.GlobalAveragePooling2D()(z4_c)

#flatten_layer = Flatten()(avg_pool_layer)

fc_layer = tf.keras.layers.Dense(10, activation='softmax')(avg_pool_layer)


model = tf.keras.models.Model(inputs=inputs, outputs= fc_layer)


here
here
here


In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 32)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 32)   128         activation[0][0]                 
______________________________________________________________________________________________

In [0]:
# from tensorflow.keras.utils import plot_model
# plot_model(model, to_file='model.png')

In [0]:
# #from one_cycle_lr import LRFinder
# from one_cycle_lr_tf import LRFinder
# 

#Best LR would be 0.01

In [0]:
# def truncate(n, decimals=0):
#     multiplier = 10 ** decimals
#     return int(n * multiplier) / multiplier

In [0]:
## for 24 epochs only

MAX_LR= 0.05
base_lr = 0.01

def lr_func(epoch,lr):
  lr = base_lr
  max_lr = MAX_LR
  
  if(epoch == 0):
    lr = base_lr
  elif(epoch>0 and epoch<11):
    lr += (max_lr-base_lr)*(epoch)/11
  else:
    lr = max_lr - (max_lr-base_lr)*(epoch-11)/18
  print("final lr ",round(lr,5))
  return round(lr,5)
  

In [0]:
opt = tf.keras.optimizers.SGD(momentum=0.9,nesterov=True)
model.compile(optimizer=opt , loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
# num_samples= train_features.shape[0]
# batch_size =128
# num_epoch=24
# max_lr=0.1

# from one_cycle_lr_tf import OneCycleLR

# lr_manager = OneCycleLR(num_samples, num_epoch, batch_size, max_lr,
#                         end_percentage=0.1, scale_percentage=None,
#                         maximum_momentum=0.95, minimum_momentum=0.85)

# opt = tf.keras.optimizers.SGD()
# model.compile(optimizer=opt , loss='categorical_crossentropy', metrics=['accuracy'])
                        


In [16]:
filepath = "Resnet-13-test1.hdf5"
from tensorflow.keras.callbacks import ModelCheckpoint,LearningRateScheduler

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')


##Train the model
model_info = model.fit_generator(train_generator,
                                 steps_per_epoch=np.ceil(50000/128), epochs=30,  
                                 validation_data = test_generator, verbose=1,callbacks=[checkpoint,LearningRateScheduler(lr_func, verbose=1)])


# model_info = model.fit_generator(train_generator,
#                                  steps_per_epoch=np.ceil(50000/128), epochs=24,  
#                                  validation_data = test_generator, verbose=1,callbacks=[checkpoint,lr_manager])


final lr  0.01

Epoch 00001: LearningRateScheduler reducing learning rate to 0.01.
Epoch 1/24
390/391 [============================>.] - ETA: 0s - loss: 1.4346 - acc: 0.4958
Epoch 00001: val_acc improved from -inf to 0.58420, saving model to Resnet-13-test1.hdf5
391/391 [==============================] - 59s 150ms/step - loss: 1.4341 - acc: 0.4959 - val_loss: 1.1749 - val_acc: 0.5842
final lr  0.01364

Epoch 00002: LearningRateScheduler reducing learning rate to 0.01364.
Epoch 2/24
390/391 [============================>.] - ETA: 0s - loss: 1.0018 - acc: 0.6430
Epoch 00002: val_acc improved from 0.58420 to 0.62990, saving model to Resnet-13-test1.hdf5
391/391 [==============================] - 50s 128ms/step - loss: 1.0015 - acc: 0.6431 - val_loss: 1.0829 - val_acc: 0.6299
final lr  0.01727

Epoch 00003: LearningRateScheduler reducing learning rate to 0.01727.
Epoch 3/24
390/391 [============================>.] - ETA: 0s - loss: 0.8313 - acc: 0.7090
Epoch 00003: val_acc improved from 0.

In [0]:
##Train the model
model_info = model.fit_generator(train_generator,
                                 steps_per_epoch=np.ceil(50000/128), epochs=6,  
                                 validation_data = test_generator, verbose=1,callbacks=[checkpoint,LearningRateScheduler(lr_func, verbose=1)])



final lr  0.01

Epoch 00001: LearningRateScheduler reducing learning rate to 0.01.
Epoch 1/6
390/391 [============================>.] - ETA: 0s - loss: 0.1673 - acc: 0.9402
Epoch 00001: val_acc improved from 0.90180 to 0.91580, saving model to Resnet-13-test1.hdf5
391/391 [==============================] - 50s 129ms/step - loss: 0.1672 - acc: 0.9403 - val_loss: 0.2788 - val_acc: 0.9158
final lr  0.01364

Epoch 00002: LearningRateScheduler reducing learning rate to 0.01364.
Epoch 2/6
390/391 [============================>.] - ETA: 0s - loss: 0.1633 - acc: 0.9422
Epoch 00002: val_acc did not improve from 0.91580
391/391 [==============================] - 50s 128ms/step - loss: 0.1634 - acc: 0.9421 - val_loss: 0.3012 - val_acc: 0.9100
final lr  0.01727

Epoch 00003: LearningRateScheduler reducing learning rate to 0.01727.
Epoch 3/6
390/391 [============================>.] - ETA: 0s - loss: 0.1666 - acc: 0.9412
Epoch 00003: val_acc did not improve from 0.91580
391/391 [====================

In [0]:
lr_func